In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import json
from keras.preprocessing.text import Tokenizer
import numpy as np 
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [2]:
with open("training_data.jsonl") as f:
    training_data = json.load(f)
with open("test_data.jsonl") as f:
    test_data = json.load(f)
with open("validation_data.jsonl") as f:
    valid_data = json.load(f)

In [3]:
train_data = []

for i in range(len(training_data)):
    train_data.append(training_data[i][0][0] + ' ' + training_data[i][0][2])
    
print(len(train_data))

1386


In [4]:
val_data = []

for i in range(len(valid_data)):
    val_data.append(valid_data[i][0][0] + ' ' + valid_data[i][0][2])
    
print(len(val_data))

175


In [5]:
REPLACE_BY_SPACE = re.compile('[/(){}\[\]\|@,;.-_-—]')

def clean_text(text):
    
    text = text.lower()
    text = REPLACE_BY_SPACE.sub(' ', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

train_data = list(map(clean_text, train_data))
val_data = list(map(clean_text, val_data))

print(len(train_data))
print(len(val_data))

1386
175


In [6]:
diction = {}
for i in training_data:
    diction[i[1]] = 1
    
print(len(diction))

j = 1
for i in diction:
    diction[i] = j
    j += 1

print(diction)

20
{'generation': 1, 'sentiment analysis, stylistic analysis, and argument mining': 2, 'information extraction, retrieval and text mining': 3, 'resources and evaluation': 4, 'machine learning for nlp': 5, 'interpretability and analysis of models for nlp': 6, 'question answering': 7, 'linguistic theories, cognitive modeling and psycholinguistics': 8, 'discourse and pragmatics': 9, 'machine translation and multilinguality': 10, 'ethics in nlp': 11, 'summarization': 12, 'tagging, chunking, syntax and parsing': 13, 'dialogue and interactive systems': 14, 'semantics': 15, 'computational social science, social media and cultural analytics': 16, 'nlp applications': 17, 'language grounding to vision, robotics and beyond': 18, 'phonology, morphology and word segmentation': 19, 'speech and multimodality': 20}


In [7]:
train_labels = []
for i in range(len(training_data)):
    train_labels.append(diction[training_data[i][1]])
    
print(len(train_labels))

val_labels = []
for i in range(len(valid_data)):
    val_labels.append(diction[valid_data[i][1]])
    
print(len(val_labels))

1386
175


In [8]:
y_train = tf.convert_to_tensor(train_labels, dtype = tf.int64)
y_val = tf.convert_to_tensor(val_labels, dtype = tf.int64)

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
tokenizer.fit_on_texts(val_data)
print("Number of unique tokens", len(tokenizer.word_index))

Number of unique tokens 9900


In [10]:
X_train = tokenizer.texts_to_sequences(train_data)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train)

X_val = tokenizer.texts_to_sequences(val_data)
X_val = tf.keras.preprocessing.sequence.pad_sequences(X_val, maxlen = 185)

print('Train data tensor shape', X_train.shape)
print('Validation data tensor shape', X_val.shape)

Train data tensor shape (1386, 185)
Validation data tensor shape (175, 185)


In [11]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(21, activation = 'softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 185, 100)          990100    
                                                                 
 spatial_dropout1d (SpatialD  (None, 185, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 21)                2121      
                                                                 
Total params: 1,072,621
Trainable params: 1,072,621
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
epochs = 30
batch_size = 64

history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (X_val,y_val))

Epoch 1/30
22/22 [==============================] - 7s 211ms/step - loss: 2.9720 - accuracy: 0.1068 - val_loss: 2.8255 - val_accuracy: 0.1086
Epoch 2/30
22/22 [==============================] - 4s 175ms/step - loss: 2.8185 - accuracy: 0.1126 - val_loss: 2.8011 - val_accuracy: 0.1771
Epoch 3/30
22/22 [==============================] - 4s 174ms/step - loss: 2.7680 - accuracy: 0.1566 - val_loss: 2.7609 - val_accuracy: 0.1086
Epoch 4/30
22/22 [==============================] - 4s 194ms/step - loss: 2.5996 - accuracy: 0.1818 - val_loss: 2.5970 - val_accuracy: 0.1600
Epoch 5/30
22/22 [==============================] - 4s 203ms/step - loss: 2.3041 - accuracy: 0.2994 - val_loss: 2.5040 - val_accuracy: 0.2114
Epoch 6/30
22/22 [==============================] - 4s 202ms/step - loss: 2.0435 - accuracy: 0.4048 - val_loss: 2.6471 - val_accuracy: 0.1943
Epoch 7/30
22/22 [==============================] - 4s 202ms/step - loss: 1.8243 - accuracy: 0.5245 - val_loss: 2.4596 - val_accuracy: 0.2114
Epoch 